In [ ]:

from cltk.corpus.utils.importer import CorpusImporter


my_latin_downloader = CorpusImporter('latin')
my_latin_downloader.list_corpora

In [ ]:
my_latin_downloader.import_corpus('latin_text_latin_library')

In [ ]:
from cltk.corpus.utils.importer import CorpusImporter
corpus_importer = CorpusImporter('latin')
corpus_importer.import_corpus('latin_models_cltk')

In [54]:
import os
import re
from cltk.stem.latin.j_v import JVReplacer
from cltk.tokenize.word import WordTokenizer
from cltk.stem.lemma import LemmaReplacer
from cltk.stop.latin import STOPS_LIST



directory = 'texts/non-comedy/poetry'
dir_list = os.listdir(directory)

for dir in dir_list:
    file_path = directory + '/' + dir
    with open(file_path, encoding='utf-8-sig') as f:
        text = f.read()
    
    jv_replacer = JVReplacer()
    text = jv_replacer.replace(text.lower())
    # text = re.sub(r'\{(.|\n)+?\}', '', text)
    # text = re.sub(r'\d[\d-]*?', '', text)    
    text = re.sub(r'[A-Z]+?\.?', '', text)
    # text = re.sub(r'<(.|\n)+?>', '', text)
    # text = re.sub(r'\[(.|\n)+?\]', '', text)
    # text = re.sub(r'\((.|\n)+?\)', '', text)
    text = re.sub(r'[-—*]', '', text)

    word_tokenizer = WordTokenizer('latin')
    text_tokens = word_tokenizer.tokenize(text)
    text_tokens = [x for x in text_tokens if x not in 
                   ['.', ',', ':', ';', '?', '!'] 
                   and x]
    
    lemmatizer = LemmaReplacer('latin')
    lemmata = lemmatizer.lemmatize(text_tokens)
    
    with open('texts/lemmatized/non-comedy/poetry/' + os.path.splitext(dir)[0] + '_lemmatized' + os.path.splitext(dir)[1], 'w', encoding='utf-8') as f:
        f.write(' '.join(lemmata))

In [62]:
import json

directory = 'texts/lemmatized/comedy'
dir_list = os.listdir(directory)

non_comedy_poetry_lemmata = []
for dir in dir_list:
    file_path = directory + '/' + dir
    print(file_path)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    text = text.split()
    non_comedy_poetry_lemmata.append(text)

with open('lemmata/comedy.json', 'w', encoding='utf-8') as f:
    json.dump(non_comedy_poetry_lemmata, f, indent=4)
    


In [32]:
from cltk.stop.latin import STOPS_LIST
import json
from collections import Counter
from pprint import pprint

with open('lemmata/comedy.json', encoding='utf-8') as f:
    comedy_lemmata = json.loads(f.read())[0]
with open('lemmata/non-comedy_poetry.json', encoding='utf-8') as f:
    non_comedy_lemmata = json.loads(f.read())[0]
with open('lemmata/non-comedy_prose.json', encoding='utf-8') as f:
    non_comedy_lemmata.extend(json.loads(f.read())[0])
comedy_lemmata = [x for x in comedy_lemmata if x not in ['amph', 'sos', 'merc', 'alc']]

# comedy_lemmata = [x for x in comedy_lemmata if x not in STOPS_LIST]
# non_comedy_lemmata = [x for x in non_comedy_lemmata if x not in STOPS_LIST]

comedy_amount = len(comedy_lemmata)
non_comedy_amount = len(non_comedy_lemmata)
for i in Counter(comedy_lemmata).most_common(10):
    print(i[0], round(i[1]*(10**6)/9801), 'ipm')
print()
for i in Counter(non_comedy_lemmata).most_common(10):
    print(i[0], round(i[1]*(10**6)/35234), 'ipm')

    

In [34]:
pprint(Counter(comedy_lemmata).most_common(100))
pprint(Counter(non_comedy_lemmata).most_common(100))

In [73]:
import re
from _collections import OrderedDict
from cltk.tag.pos import POSTag
tagger = POSTag('latin')


comedy_freq_dict = {}
non_comedy_freq_dict = {}

for lemma in comedy_lemmata:
    if lemma not in comedy_freq_dict:
        comedy_freq_dict[lemma] = 0
    comedy_freq_dict[lemma] += 1
for i in comedy_freq_dict:
    comedy_freq_dict[i] = round(comedy_freq_dict[i]*(10**6)/9801, 3)
    
for i in comedy_freq_dict:
    lemma = re.sub('\d', '', i)
    number = comedy_freq_dict[i]
    comedy_freq_dict[i] = (None, comedy_freq_dict[i])
    analysis = tagger.tag_ngram_123_backoff(lemma)[0][1]
    if analysis:
        if not analysis.startswith('-'):
            comedy_freq_dict[i] = (analysis[0], number)        

comedy_freq_dict = list(comedy_freq_dict.items())
comedy_freq_dict.sort(key=lambda x: x[1][1], reverse=True)
comedy_freq_dict = OrderedDict(comedy_freq_dict)

for lemma in non_comedy_lemmata:
    if lemma not in non_comedy_freq_dict:
        non_comedy_freq_dict[lemma] = 0
    non_comedy_freq_dict[lemma] += 1
for i in non_comedy_freq_dict:
    non_comedy_freq_dict[i] = round(non_comedy_freq_dict[i]*(10**6)/35234, 3)
    
for i in non_comedy_freq_dict:
    lemma = re.sub('\d', '', i)
    number = non_comedy_freq_dict[i]
    non_comedy_freq_dict[i] = (None, non_comedy_freq_dict[i])
    analysis = tagger.tag_ngram_123_backoff(lemma)[0][1]
    if analysis:
        if not analysis.startswith('-'):
            non_comedy_freq_dict[i] = (analysis[0], number)            
    
non_comedy_freq_dict = list(non_comedy_freq_dict.items())
non_comedy_freq_dict.sort(key=lambda x: x[1][1], reverse=True)
non_comedy_freq_dict = OrderedDict(non_comedy_freq_dict)
print(non_comedy_freq_dict)

    
with open('comedy_freq_dict.json', 'w', encoding='utf-8') as f:
    json.dump(comedy_freq_dict, f, indent=4)

with open('non_comedy_freq_dict.json', 'w', encoding='utf-8') as f:
    json.dump(non_comedy_freq_dict, f, indent=4)


In [20]:
import json

with open('comedy_freq_dict.json', encoding='utf-8') as f:
    comedy_freq_dict = json.loads(f.read())
with open('non_comedy_freq_dict.json', encoding='utf-8') as f:
    non_comedy_freq_dict = json.loads(f.read())

comedy_pos_dict = {}
for i in comedy_freq_dict:
    if comedy_freq_dict[i][0]:
        if comedy_freq_dict[i][0] not in comedy_pos_dict:
            comedy_pos_dict[comedy_freq_dict[i][0]] = 0
        comedy_pos_dict[comedy_freq_dict[i][0]] += comedy_freq_dict[i][1]
    else:
        if 'NA' not in comedy_pos_dict:
            comedy_pos_dict['NA'] = 0
        comedy_pos_dict['NA'] += comedy_freq_dict[i][1]
print(comedy_pos_dict)

non_comedy_pos_dict = {}
for i in non_comedy_freq_dict:
    if non_comedy_freq_dict[i][0]:
        if non_comedy_freq_dict[i][0] not in non_comedy_pos_dict:
            non_comedy_pos_dict[non_comedy_freq_dict[i][0]] = 0
        non_comedy_pos_dict[non_comedy_freq_dict[i][0]] += non_comedy_freq_dict[i][1]
    else:
        if 'NA' not in non_comedy_pos_dict:
            non_comedy_pos_dict['NA'] = 0
        non_comedy_pos_dict['NA'] += non_comedy_freq_dict[i][1]
print(non_comedy_pos_dict)

{'NA': 333027.01100001443, 'P': 48158.35, 'V': 108560.34500000006, 'D': 62646.66000000002, 'A': 69788.78600000002, 'U': 11019.284000000001, 'N': 101418.18700000002, 'C': 4081.216000000001, 'R': 6121.820999999995, 'E': 1428.425, 'M': 1428.424, 'T': 102.03}
{'P': 39961.40099999999, 'NA': 383635.6969999377, 'U': 22591.814, 'V': 64710.22399999996, 'A': 80036.32600000006, 'D': 39280.24999999993, 'N': 138020.10400000325, 'R': 4541.076999999995, 'C': 2412.443, 'M': 1560.993, 'E': 1078.5049999999999, 'T': 851.4509999999996}
